In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from scipy.spatial import distance
import json 

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
postcodes = pd.read_csv("../data/curated/unique_postcodes.csv", header=None, names = ['postcode'])
latlongs = pd.read_csv("../data/raw/external/australian_postcodes.csv")

In [42]:
latlongs = latlongs.loc[latlongs['state']  == 'VIC']

In [4]:
merged = postcodes.merge(latlongs, on = 'postcode', how = "left")
postcode_latlong = merged[['postcode', 'lat', 'long']]

In [43]:
postcode_latlong = postcode_latlong.drop_duplicates(subset = ['postcode'])

In [40]:
# calculate distance to each postcode   

lat = postcode_latlong['lat'][0]
long = postcode_latlong['long'][0]

coord = [lat, long]

lat1 = postcode_latlong['lat'][2]
long1 = postcode_latlong['long'][2]
coord1 = [lat1, long1]

postcode_dict = {}
for i in range(0, len(postcode_latlong)):
    inner_dict = {}
    current_postcode = postcode_latlong.iloc[i]
    curr_coords = [current_postcode['lat'], current_postcode['long']]

    for j in range(0, len(postcode_latlong)):
        if j == i:
            continue
        else:
            # calculate euclidean distance between current postcode and all others
            next_postcode = postcode_latlong.iloc[j]
            next_coords = [next_postcode['lat'], next_postcode['long']]

            inner_dict[int(next_postcode['postcode'])] = distance.euclidean(curr_coords, next_coords)

    df = pd.DataFrame(list(inner_dict.items()), columns = ['postcode', 'distance'])
    df = df.sort_values('distance')
    ten_closest = df.head(10)

    postcode_dict[int(current_postcode['postcode'])] = dict(zip(ten_closest['postcode'], ten_closest['distance']))



In [74]:
with open("../data/curated/postcode_dists.json", "w") as outfile: 
    json.dump(postcode_dict, outfile)

In [58]:
# read in growth rate, affordability and liveability csvs and combine for each postcode 
growth_rate_df = pd.read_csv("../data/curated/growthrate.csv")
affordability_df = pd.read_csv("../data/curated/affordability.csv")
livability_df = pd.read_csv("../data/curated/liveability.csv")


In [61]:
growth_rate_df.columns = ['Postcode', 'Growth Rate']

In [62]:
livability_df.dropna()

,Unnamed: 0,Postcode,Standardised Liveability
0,0,3000,0.132226
1,1,3002,0.167572
2,2,3003,0.155316
3,3,3004,0.225321
4,4,3006,0.292377
...,...,...,...
685,685,3981,0.311540
686,686,3984,0.653477
687,687,3987,0.849286
692,692,3995,0.447956


In [63]:
affordability_df

,Unnamed: 0,Postcode,Standardised Affordability
0,0,3000,0.062203
1,1,3002,0.031659
2,2,3003,0.588788
3,3,3004,0.044945
4,4,3006,0.000000
...,...,...,...
221,221,3975,0.769464
222,222,3976,0.679672
223,223,3977,0.663470
224,224,3978,0.670703


In [69]:
merged = growth_rate_df.merge(affordability_df, on='Postcode').merge(livability_df, on = 'Postcode')
final_df = merged.drop(['Unnamed: 0_x', 'Unnamed: 0_y'], axis = 1)

In [76]:
final_df.to_csv('../data/curated/growthrate_afford_live.csv')